In [2]:
import torch
import torch.nn as nn

/home/ehsan/miniconda3/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# define adjacency matrix 

In [4]:
# This matrix shows that the GCN doesn't work on heterophilic graphs.

# A = torch.tensor([
#     [0, 1, 0, 1],
#     [1, 0, 1, 0],
#     [0, 1, 0, 1],
#     [1, 0, 1, 0]
# ], dtype=torch.float)

In [5]:
A = torch.tensor([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
], dtype=torch.float)

In [6]:
# Node's Features or Feature Matrix 

In [7]:
X = torch.tensor([
    [1, 0],  
    [0, 1],  
    [1, 1],  
    [0, 0]   
], dtype=torch.float)

In [8]:
labels = torch.tensor([0, 1, 0, 1], dtype=torch.long)

In [9]:
# add self loop

In [10]:
I = torch.eye(A.shape[0])
A_hat = A+I

In [11]:
# Degree matrix

In [12]:
degrees = A_hat.sum(1)
degrees

tensor([2., 2., 2., 2.])

In [13]:
D_hat_inv_sqrt = torch.diag(degrees.pow(-0.5))
D_hat_inv_sqrt

tensor([[0.7071, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.7071, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.7071, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.7071]])

In [14]:
support = D_hat_inv_sqrt @ A_hat @ D_hat_inv_sqrt
support

tensor([[0.5000, 0.0000, 0.5000, 0.0000],
        [0.0000, 0.5000, 0.0000, 0.5000],
        [0.5000, 0.0000, 0.5000, 0.0000],
        [0.0000, 0.5000, 0.0000, 0.5000]])

In [15]:
# define GCN 

In [16]:
class GCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GCN, self).__init__()
        self.layer1 = nn.Linear(in_features, hidden_features, bias=False)
        self.layer2 = nn.Linear(hidden_features, out_features, bias=False)
        self.relu = nn.ReLU()
        
    def forward(self, x, supput_matrix):
        x = self.layer1(x)
        x = torch.mm(supput_matrix, x)
        x = self.relu(x)
        
        x = self.layer2(x)
        x = torch.mm(supput_matrix, x)
        return x

In [17]:
model = GCN(in_features=2, hidden_features=4, out_features=2)

In [18]:
import torch.optim as optim

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [20]:
criterion = nn.CrossEntropyLoss()

In [23]:
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    
    out = model(X, support)
    loss = criterion(out, labels)
    
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
      print(f"Epoch {epoch} | Loss {loss.item():.4f}")  

Epoch 0 | Loss 0.3617
Epoch 10 | Loss 0.3593
Epoch 20 | Loss 0.3574
Epoch 30 | Loss 0.3556
Epoch 40 | Loss 0.3542
Epoch 50 | Loss 0.3531
Epoch 60 | Loss 0.3523
Epoch 70 | Loss 0.3517
Epoch 80 | Loss 0.3513
Epoch 90 | Loss 0.3512
Epoch 100 | Loss 0.3510
Epoch 110 | Loss 0.3506
Epoch 120 | Loss 0.3502
Epoch 130 | Loss 0.3499
Epoch 140 | Loss 0.3496
Epoch 150 | Loss 0.3493
Epoch 160 | Loss 0.3491
Epoch 170 | Loss 0.3489
Epoch 180 | Loss 0.3487
Epoch 190 | Loss 0.3486


In [24]:
model.eval()
pred = model(X, support).argmax(dim=1)
print("\nLabels:     ", labels.tolist())
print("Predictions:", pred.tolist())


Labels:      [0, 1, 0, 1]
Predictions: [0, 1, 0, 1]
